In [6]:
#need to update third element to corresponding semester
targets = {"320", "data programming", "s22", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "project"}
def match(repo): 
    repo = repo.lower()
    if (repo.find("cs320-lab2") == -1 and repo.find("cs320_lab2") == -1):
        for word in targets: 
            if repo.find(word) != -1: 
                return repo

In [7]:
import pandas
from github import Github

usernames = pandas.read_csv("github.csv") #first upload github.csv file with past semesters info to current directory
g = Github("") # add your OAuth token here, use link below to create
#https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token

In [8]:
#old approach, only looks at repository names and not the content
not_found = []
suspect = []

for username in usernames['What is your GitHub username?']: 
    try:
        user = g.get_user(username)
        for repo in user.get_repos():
            mat = match(repo.full_name)
            if mat is not None: 
                suspect.append(mat)
    except: 
        not_found.append(username)

In [3]:
#new improved approach, looks at repo name then opens repo looking for notebook files following our submittion naming convention
class Found(Exception): pass
not_found = []
suspect = []
projects = ["p1.ipynb","p2.ipynb","p3.ipynb","p4.ipynb","p5.ipynb","p6.ipynb","p7.ipynb"]
for username in usernames['What is your GitHub username?']: 
    try:
        user = g.get_user(username)
        for repo in user.get_repos():
            mat = match(repo.full_name)
            if mat is not None:
                try:
                    #get repo content from root directory
                    contents  = repo.get_contents("")
                    while contents:
                        file_content = contents.pop(0)
                        if file_content.type == "dir": #checks if directory
                            contents.extend(repo.get_contents(file_content.path)) #opens directory
                        else:
                            for project in projects:
                                if file_content.path.endswith(project):
                                    suspect.append(mat)
                                    raise Found
                except Found:
                    break
                except:
                    pass
    except: 
        not_found.append(username)

In [4]:
print(not_found)

['howardxl', 'annamelnyk4@gmail.com', 'jinacho', 'eric970412', 'piettec', 'gervolino']


In [5]:
#writes output to file
outputfile = open('githubResults.txt', 'w')
for i in suspect: 
    outputfile.write("https://github.com/" + i + "\n")
outputfile.close()